In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

24/04/06 10:56:23 WARN Utils: Your hostname, Luiss-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.100.206 instead (on interface en1)
24/04/06 10:56:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/06 10:56:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/06 10:56:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/06 10:56:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

24/04/06 10:57:08 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
my_log_reg_model = LogisticRegression()

In [7]:
fitted_logreg = my_log_reg_model.fit(my_data)

24/04/06 10:58:26 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [8]:
log_summary = fitted_logreg.summary

In [10]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [11]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514870...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198872...|[6.76550379997544...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234669...|[4.87842678719363...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012715...|[4.62137287300564...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874693...|[1.81823629114105...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504180...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212819...|[6.97903542820426...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446273...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606545...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [12]:
lr_train, lr_test = my_data.randomSplit([0.7, 0.3])

In [13]:
final_model = LogisticRegression()

In [15]:
fit_final = final_model.fit(lr_train)

In [16]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [17]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[17.1696565317755...|[0.99999996506080...|       0.0|
|  0.0|(692,[122,123,124...|[16.8834419218296...|[0.99999995348271...|       0.0|
|  0.0|(692,[123,124,125...|[28.1719208640618...|[0.99999999999941...|       0.0|
|  0.0|(692,[124,125,126...|[23.6447201312349...|[0.99999999994614...|       0.0|
|  0.0|(692,[126,127,128...|[24.1540980571779...|[0.99999999996764...|       0.0|
|  0.0|(692,[126,127,128...|[25.2571189558742...|[0.99999999998926...|       0.0|
|  0.0|(692,[128,129,130...|[18.6687156953240...|[0.99999999219667...|       0.0|
|  0.0|(692,[151,152,153...|[31.1763226062026...|[0.99999999999997...|       0.0|
|  0.0|(692,[234,235,237...|[-0.1556809432588...|[0.46115818183993...|       1.0|
|  1.0|(692,[100

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [19]:
my_eval = BinaryClassificationEvaluator()

In [20]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [21]:
my_final_roc

0.9500000000000001